# Load Libraries

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import signal

img_orig = mpimg.imread('132148_header.jpg')
img = img_orig[:256, :256, 1] / 255
plt.imshow(img)

# Generate Odd White Noise

In [ ]:
noise = np.zeros((img.shape[0], img.shape[1]), dtype = 'float')
for n in range(img.shape[0]):
    for m in range(n):
        if(n == 0 and m == 0): #make matrix symmetric
            noise[n][m] = 0
        else:
            noise[n][m] = np.random.uniform(low = -np.pi, high = np.pi)
            noise[m][n] = noise[n][m]

# Basic RPN

In [ ]:
x = img
x = np.fft.fft2(x)
x = x * np.exp(1j*noise)
x = np.fft.ifft2(x)
print('Max of the imaginary part of x: ', np.max(np.imag(x)))
plt.imshow(np.real(x))

# Basic RPN w/ Color

In [ ]:
img = img_orig[:256, :256, :] / 255
print(img.shape)
plt.figure()
plt.imshow(img)
for n in range(3):
    x = img[:,:,n]
    x = np.fft.fft2(x)
    x = x * np.exp(1j*noise)
    img[:,:,n] = np.fft.ifft2(x)
plt.figure()
plt.imshow(np.real(img))

# Fixed RPN (Same Size)


In [ ]:
#filter
filter = np.zeros((3,3), dtype = 'float')
filter[1,1] = -4
filter[0,1] = 1
filter[1,2] = 1
filter[1,0] = 1
filter[2,1] = 1
print(filter)

# Create Periodic Laplacian

In [ ]:
img = img_orig[:256, :256, 1] / 255
(M,N) = img.shape
l = np.zeros((M,N), dtype = 'float')

# interior pixels
l[1:M-1,1:N-1] = signal.convolve2d(img, filter, mode = 'valid')

# edge pixels, but not corners
for m in range(1,M - 1):  
    l[m,0] = img[m-1,0] + img[m+1,0] + img[m,1] - 3*img[m,0]
    l[m,N-1] = img[m-1,N-1] + img[m+1,N-1] + img[m,N-2] - 3*img[m,N-1]
for n in range(1,N-1):
    l[0,n] = img[1,n] + img[0,n-1] + img[0,n+1] - 3*img[0,n]
    l[M-1,n] = img[M-2,n] + img[M-1,n-1] + img[M-1,n+1] - 3*img[M-1,n]

# corner pixels
l[0,0] = img[0,1] + img[1,0] - 2*img[0,0]
l[0,N-1] = img[0,N-2] + img[1,N-1] - 2*img[0,N-1]
l[M-1,0] = img[M-1,1] + img[M-2,0] - 2*img[M-1,0]
l[M-1,N-1] = img[M-1,N-2] + img[M-2,N-1] - 2*img[M-1,N-1]

l_hat = np.fft.fft2(l)
p_hat = np.zeros((M,N), dtype = 'complex')
for m in range(0,M): 
    for n in range(0,N):
        if(n == 0 and m == 0):
            p_hat[m,n] = np.sum(img)
        else:
            alpha = 4 - 2 * np.cos(2*n*np.pi/N) - 2 * np.cos(2*m*np.pi/M)
            p_hat[m,n] = (-1/alpha) * l_hat[m,n]
p_w_noise = p_hat * np.exp(1j*noise)
texture = np.fft.ifft2(p_w_noise)
plt.imshow(np.real(texture))

In [ ]:
# For testing purposes

# Get the periodic component
p = np.fft.ifft2(p_hat)
print('Max of the imaginary part of p: ', np.max(np.abs(np.imag(p))))

# Plot
plt.figure(figsize = (20,5))
plt.subplot(1,3,1)
plt.imshow(img)
plt.grid(False)
plt.colorbar()
plt.title('Original image')
plt.subplot(1,3,2)
plt.imshow(np.real(p))
plt.grid(False)
plt.colorbar()
plt.title('Periodic component')
plt.subplot(1,3,3)
plt.imshow(img - np.real(p))
plt.grid(False)
plt.colorbar()
plt.title('Smooth component');


# RPN w/ Color


In [ ]:
#filter
filter = np.zeros((3,3), dtype = 'float')
filter[1,1] = -4
filter[0,1] = 1
filter[1,2] = 1
filter[1,0] = 1
filter[2,1] = 1
print(filter)

In [ ]:
img = img_orig[:256, :256, :]/255
plt.figure()
plt.imshow(img)
(M,N,RGB) = img.shape
l = np.zeros((M,N,RGB), dtype = 'complex') #only one tensor. more variables = expensive
for i in range(RGB):
    #interior
    l[1:M-1,1:N-1,i] = signal.convolve2d(img[:,:,i], filter, mode = 'valid')
    #edge pixels, but not corners
    for m in range(1,M - 1):  
        l[m,0,i] = img[m-1,0,i] + img[m+1,0,i] + img[m,1,i] - 3*img[m,0,i]
        l[m,N-1,i] = img[m-1,N-1,i] + img[m+1,N-1,i] + img[m,n-2,i] - 3*img[m,N-1,i]
    for n in range(1,N-1):
        l[0,n,i] = img[1,n,i] + img[1,n-1,i] + img[0,n+1,i] - 3*img[0,n,i]
        l[M-1,n,i] = img[M-2,n,i] + img[M-1,n-1,i] + img[M-1,n+1,i] - 3*img[M-1,n,i]
    #corner pixels
    l[0,0,i] = img[0,1,i] + img[1,0,i] - 2*img[0,0,i]
    l[0,N-1,i] = img[0,N-2,i] + img[1,N-1,i] - 2*img[0,N-1,i]
    l[M-1,0,i] = img[M-1,1,i] + img[M-2,0,i] - 2*img[M-1,0,i]
    l[M-1,N-1,i] = img[M-1,N-2,i] + img[M-2,N-1,i] - 2*img[M-1,N-1,i]
    l[:,:,i] = np.fft.fft2(l[:,:,i])
    
    #create periodic component
    for m in range(1,M-1): 
        for n in range(1,N-1):
            if(n == 0 and m == 0):
                l[m,n,i] = np.sum(img[:,:,i])
            else:
                alpha = 4 - 2 * np.cos(2*n*np.pi/N) - 2 * np.cos(2*m*np.pi/M)
                l[m,n,i] = -1/alpha * l[m,n,i]
    l[:,:,i] = l[:,:,i] * np.exp(1j*noise)
    l[:,:,i] = np.fft.ifft2(l[:,:,i])
plt.figure()
plt.imshow(np.real(l))
    


In [ ]:
plt.figure()
plt.imshow(img[:,:,0])
plt.figure()
plt.imshow(np.real(l[:,:,0]))

# RPN for General Size w/ No color

In [ ]:
#no color
print('Original image pixels: ')
print(img_orig[0:5,0:5,0])
img = img_orig[:128,:256,0]/255
print(img[0:5,0:5])

#noise
noise = np.zeros((img.shape[0],img.shape[1]), dtype = 'float')
for n in range(img.shape[0]):
    for m in range(img.shape[1]):
        if(n ==0 and m ==0): 
          noise[n][m] = 0
        else:
          noise[n][m] = np.random.uniform(low = -np.pi, high = np.pi)
#           noise[m][n] = noise[n][m]

#filter
filter = np.zeros((3,3), dtype = 'float')
filter[1,1] = -4
filter[0,1] = 1
filter[1,2] = 1
filter[1,0] = 1
filter[2,1] = 1
print(filter)


In [ ]:
(M,N) = img.shape
l = np.zeros((M,N), dtype = 'complex')
l[1:M-1,1:N-1] = signal.convolve2d(img, filter, mode = 'valid')
#edge pixels, but not corners
for m in range(1,M - 1):  
    l[m,0] = img[m-1,0] + img[m+1,0] + img[m,1] - 3*img[m,0]
    l[m,N-1] = img[m-1,N-1] + img[m+1,N-1] + img[m,N-2] - 3*img[m,N-1]
for n in range(1,N-1):
    l[0,n] = img[1,n] + img[1,n-1] + img[0,n+1] - 3*img[0,n]
    l[M-1,n] = img[M-2,n] + img[M-1,n-1] + img[M-1,n+1] - 3*img[M-1,n]

#corner pixels
l[0,0] = img[0,1] + img[1,0] - 2*img[0,0]
l[0,N-1] = img[0,N-2] + img[1,N-1] - 2*img[0,N-1]
l[M-1,0] = img[M-1,1] + img[M-2,0] - 2*img[M-1,0]
l[M-1,N-1] = img[M-1,N-2] + img[M-2,N-1] - 2*img[M-1,N-1]

l = np.fft.fft2(l)
for m in range(1,M-1): #interior pixels
    for n in range(1,N-1):
        if(n == 0 and m == 0):
            l[m][n] = np.sum(img)
        else:
            alpha = 4 - 2 * np.cos(2*n*np.pi/N) - 2 * np.cos(2*m*np.pi/M)
            l[m][n] = -1/alpha * l[m][n]
l = l * np.exp(1j*noise)
l = np.fft.ifft2(l)
plt.imshow(np.real(l))